In [7]:
import pandas as pd
import json

In [8]:
cases = pd.read_csv('cases.csv')
labs = pd.read_csv('labs.csv')
cases.columns
cases

,caseid,subjectid,casestart,caseend,anestart,aneend,opstart,opend,adm,dis,...,intraop_colloid,intraop_ppf,intraop_mdz,intraop_ftn,intraop_rocu,intraop_vecu,intraop_eph,intraop_phe,intraop_epi,intraop_ca
0,1,5955,0,11542,-552,10848.0,1668,10368,-236220,627780,...,0,120,0.0,100,70,0,10,0,0,0
1,2,2487,0,15741,-1039,14921.0,1721,14621,-221160,1506840,...,0,150,0.0,0,100,0,20,0,0,0
2,3,2861,0,4394,-590,4210.0,1090,3010,-218640,40560,...,0,0,0.0,0,50,0,0,0,0,0
3,4,1903,0,20990,-778,20222.0,2522,17822,-201120,576480,...,0,80,0.0,100,100,0,50,0,0,0
4,5,4416,0,21531,-1009,22391.0,2591,20291,-67560,3734040,...,0,0,0.0,0,160,0,10,900,0,2100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6383,6384,5583,0,15248,-260,15640.0,2140,14140,-215340,648660,...,0,150,0.0,0,90,0,20,0,0,0
6384,6385,2278,0,20643,-544,20996.0,2396,19496,-225600,1675200,...,0,100,0.0,0,100,0,25,30,0,300
6385,6386,4045,0,19451,-667,19133.0,3533,18233,-200460,836340,...,0,70,0.0,0,130,0,10,0,0,0
6386,6387,5230,0,12025,-550,12830.0,1730,11030,-227760,377040,...,0,120,0.0,0,50,0,0,0,0,0


In [45]:
columns_to_keep = [
    'optype', 'asa', 'sex', 'age', 'height', 'weight',
    'adm', 'dis', 'caseend', 'anestart', 'aneend',
    'iv1', 'iv2', 'aline1', 'aline2', 'cline1', 'cline2', 'ane_type'
]

output = cases[columns_to_keep].copy()

output['asa'] = output['asa'].round(2).fillna('N/A')
output['adm'] = output['adm'].round(2)
output['dis'] = output['dis'].round(2)
output['age'] = output['age'].round(1).fillna('N/A')
output['height'] = output['height'].round(1).fillna('N/A')
output['weight'] = output['weight'].round(1).fillna('N/A')
output['iv1'] = output['iv1'].fillna('N/A')
output['iv2'] = output['iv2'].fillna('N/A')
output['aline1'] = output['aline1'].fillna('N/A')
output['aline2'] = output['aline2'].fillna('N/A')
output['cline1'] = output['cline1'].fillna('N/A')
output['cline2'] = output['cline2'].fillna('N/A')
output['ane_type'] = output['ane_type'].fillna('N/A')

output['stay_duration'] = (output['dis'] - output['adm'])
output['surgery_duration'] = output['caseend']
output['anesthesia_lead_time'] = output['anestart']
output['anesthesia_duration'] = output['aneend']


# mapping onto regions


output["region"] = output["optype"]

# Drop raw timing columns if not needed in frontend
output.drop(['adm', 'dis', 'caseend', 'anestart', 'aneend'], axis=1, inplace=True)

# Convert to list of dicts for JSON
output = output.to_dict(orient='records')

output

[{'optype': 'Colorectal',
  'asa': 2.0,
  'sex': 'M',
  'age': 77.0,
  'height': 160.2,
  'weight': 67.5,
  'iv1': 'Right forearm',
  'iv2': 'N/A',
  'aline1': 'Left radial',
  'aline2': 'N/A',
  'cline1': 'N/A',
  'cline2': 'N/A',
  'ane_type': 'General',
  'stay_duration': 864000,
  'surgery_duration': 11542,
  'anesthesia_lead_time': -552,
  'anesthesia_duration': 10848.0,
  'region': 'Colorectal'},
 {'optype': 'Stomach',
  'asa': 2.0,
  'sex': 'M',
  'age': 54.0,
  'height': 167.3,
  'weight': 54.8,
  'iv1': 'Left forearm',
  'iv2': 'N/A',
  'aline1': 'N/A',
  'aline2': 'N/A',
  'cline1': 'N/A',
  'cline2': 'N/A',
  'ane_type': 'General',
  'stay_duration': 1728000,
  'surgery_duration': 15741,
  'anesthesia_lead_time': -1039,
  'anesthesia_duration': 14921.0,
  'region': 'Stomach'},
 {'optype': 'Biliary/Pancreas',
  'asa': 1.0,
  'sex': 'M',
  'age': 62.0,
  'height': 169.1,
  'weight': 69.7,
  'iv1': 'Left forearm',
  'iv2': 'N/A',
  'aline1': 'N/A',
  'aline2': 'N/A',
  'cline1'

In [46]:
# Save JSON
with open('kate_card.json', 'w') as f:
    json.dump(output, f, indent=2)

In [37]:
# columns_to_keep = [
#     'optype', 'asa', 'sex', 'age', 'height', 'weight',
#     'adm', 'dis',
#     'iv1', 'iv2', 'aline1', 'aline2', 'cline1', 'cline2', 'ane_type'
# ]
# output = cases[columns_to_keep].copy()

# output['asa'] = output['asa'].round(2).fillna('N/A')
# output['adm'] = output['adm'].round(2).fillna('N/A')
# output['dis'] = output['dis'].round(2).fillna('N/A')
# output['age'] = output['age'].round(1).fillna('N/A')
# output['height'] = output['height'].round(1).fillna('N/A')
# output['weight'] = output['weight'].round(1).fillna('N/A')
# output['iv1'] = output['iv1'].fillna('N/A')
# output['iv2'] = output['iv2'].fillna('N/A')
# output['aline1'] = output['aline1'].fillna('N/A')
# output['aline2'] = output['aline2'].fillna('N/A')
# output['cline1'] = output['cline1'].fillna('N/A')
# output['cline2'] = output['cline2'].fillna('N/A')
# output['ane_type'] = output['ane_type'].fillna('N/A')


# # Convert to list of dicts for JSON
# output = output.to_dict(orient='records')

# # Save JSON
# with open('dashboard_data.json', 'w') as f:
#     json.dump(output, f, indent=2)

adm - time before operation started
dis - time after operation started
optype - operation type
asa - tells about patient condition
iv1 - location of first iv (in all)
some tube locationsfor the rest: iv2, aline1, aline2, cline1, cline2
ane_type - anesthesia type

In [ ]:
# numeric_cols = ['adm', 'dis', 'asa']
# mode_cols = ['iv1', 'iv2', 'aline1', 'aline2', 'cline1', 'cline2', 'ane_type']
# group_col = 'optype'

# # Define function to get mode safely (returns NaN if no mode)
# def get_mode(series):
#     modes = series.mode()
#     return modes.iloc[0] if not modes.empty else None

# # Aggregate numeric by mean and categorical by mode
# agg_dict = {col: 'mean' for col in numeric_cols}
# agg_dict.update({col: get_mode for col in mode_cols})

# grouped = df.groupby(group_col).agg(agg_dict).reset_index()

# # Round numeric columns for cleaner JSON
# for col in numeric_cols:
#     grouped[col] = grouped[col].round(2)

# # Build the JSON output array
# output = []
# for _, row in grouped.iterrows():
#     output.append({
#         "name": row[group_col],
#         "adm": row['adm'],
#         "dis": row['dis'],
#         "asa": row['asa'],
#         "iv1": row['iv1'],
#         "iv2": row['iv2'],
#         "aline1": row['aline1'],
#         "aline2": row['aline2'],
#         "cline1": row['cline1'],
#         "cline2": row['cline2'],
#         "ane_type": row['ane_type']
#     })

# # Save JSON output
# with open('dashboard_data.json', 'w') as f:
#     json.dump(output, f, indent=2)